In [2]:
!pip install langchain langchain-community faiss-cpu sentence-transformers pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1

In [3]:
import os
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
import pandas as pd
from langchain.docstore.document import Document

In [4]:
vectordb_file_path = "faiss_index"

def create_vector_db():
    print("Initializing a fast, local embedding model to run on your CPU...")
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'}
    )

    csv_path = "/content/medquad_parsed_data.csv"
    chunk_size = 500

    print(f"Loading data from {csv_path} in chunks of {chunk_size}...")
    # Add error_bad_lines=False to be more robust against potential CSV errors
    reader = pd.read_csv(csv_path, chunksize=chunk_size, encoding='utf-8', on_bad_lines='skip')

    vectordb = None

    for i, chunk in enumerate(reader):
        print(f"--- Processing chunk {i+1} ---")
        chunk = chunk.dropna(subset=['question', 'answer'])
        documents = [
            Document(
                page_content=row['answer'],
                metadata={'source': row['question']}
            ) for index, row in chunk.iterrows()
        ]

        if not documents:
            print(f"Chunk {i+1} is empty after cleaning. Skipping.")
            continue

        if vectordb is None:
            print(f"Creating initial vector database from chunk {i+1}...")
            vectordb = FAISS.from_documents(documents=documents, embedding=embeddings)
        else:
            print(f"Adding chunk {i+1} to the existing vector database...")
            vectordb.add_documents(documents=documents)

    if vectordb:
        print(f"\nSaving final vector database to {vectordb_file_path}...")
        vectordb.save_local(vectordb_file_path)
        print("Vector database created and saved successfully.")
    else:
        print("No data was processed. Vector database not created.")

# Main execution block
if not os.path.exists(vectordb_file_path):
    create_vector_db()
else:
    print("Vector database already exists. If you want to recreate it, delete the 'faiss_index' folder.")


Initializing a fast, local embedding model to run on your CPU...


/tmp/ipython-input-4219545232.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

--- Processing chunk 94 ---
Adding chunk 94 to the existing vector database...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


--- Processing chunk 95 ---
Adding chunk 95 to the existing vector database...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Saving final vector database to faiss_index...
Vector database created and saved successfully.


In [5]:
# After the process is complete, run this cell to zip the folder.
!zip -r faiss_index.zip faiss_index

  adding: faiss_index/ (stored 0%)
  adding: faiss_index/index.pkl (deflated 76%)
  adding: faiss_index/index.faiss (deflated 10%)
